# census-2021

In [21]:
# PACKAGES
import geopandas as gpd
import pandas as pd
from janitor import clean_names
import altair as alt
import requests

In [17]:
# DATA
url = 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationandhouseholdestimatesenglandandwalescensus2021/census2021/census2021firstresultsenglandwales1.xlsx'
filename = '../data/external/census2021firstresultsenglandwales1.xlsx'

In [ ]:
# FETCH
r = requests.get(url)
open(filename, 'wb').write(r.content)
df = pd.read_excel(filename)

In [27]:
# P01 Census 2021: Usual resident population by sex, local authorities in England and Wales
P01 = pd.read_excel(filename,
sheet_name="P01", header=6)

In [20]:
P01[P01['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,All persons,Females,Males
1,E92000001,England,56489800,28833500,27656300
211,E12000007,London,8799800,4531500,4268300
224,E09000030,Tower Hamlets,310300,154500,155800


In [16]:
# P02 Census 2021: Usual resident population by five-year age group, local authorities in England and Wales
P02 = pd.read_excel(filename,
sheet_name="P02", header=7)

In [17]:
P02[P02['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,All persons,Aged 4 years and under\n[note 12],Aged 5 to 9 years\n[note 12],Aged 10 to 14 years\n[note 12],Aged 15 to 19 years\n[note 12],Aged 20 to 24 years\n[note 12],Aged 25 to 29 years\n[note 12],Aged 30 to 34 years\n[note 12],...,Aged 45 to 49 years\n[note 12],Aged 50 to 54 years\n[note 12],Aged 55 to 59 years\n[note 12],Aged 60 to 64 years\n[note 12],Aged 65 to 69 years\n[note 12],Aged 70 to 74 years\n[note 12],Aged 75 to 79 years\n[note 12],Aged 80 to 84 years\n[note 12],Aged 85 to 89 years\n[note 12],Aged 90 years and over\n[note 12]
1,E92000001,England,56489800,3077000,3348600,3413100,3218900,3414400,3715400,3952600,...,3602600,3907700,3806300,3256100,2767500,2796600,2038800,1427900,872200,498200
211,E12000007,London,8799800,529000,531500,535400,489400,589200,786600,808300,...,594000,573800,511600,404600,306400,269100,189700,141000,86800,50400
224,E09000030,Tower Hamlets,310300,19100,17700,17400,18400,32000,44500,40700,...,17500,13900,10800,8500,6000,4300,2800,2200,1300,700


In [18]:
# P03 Census 2021: Usual resident population by sex and five-year age group, local authorities in England and Wales
P03 = pd.read_excel(filename,
sheet_name="P03", header=7)

In [19]:
def reshape_gender_data(df):
    df.reset_index(drop=True, inplace=True)
    df = df.rename(columns=lambda x:x.replace('note 12',''))
    df = df.clean_names(remove_special=True)
    df = df.rename(columns=lambda x:x.replace('females_aged_',''))
    df = df.rename(columns=lambda x:x.replace('males_aged_',''))
    df = df.rename(columns=lambda x:x.replace('_years',''))
    df = df.rename(columns=lambda x:x.replace('_to_','-'))
    df = df.rename(columns=lambda x:x.replace('4_and_under','00-04'))
    df = df.rename(columns=lambda x:x.replace('_and_over','+'))
    df = df.rename(columns=lambda x:x.replace('5-9','05-09'))
    df = df.transpose()
    df.reset_index(inplace=True)
    df.columns = ['group','england and wales', 'london', 'Tower Hamlets']
    # df = df.iloc[::-1]
    return df

In [20]:
P03_females = reshape_gender_data(P03[P03['Area name'].isin(['Tower Hamlets', 'England', 'London'])].filter(regex=r"Females.*"))
P03_males = reshape_gender_data(P03[P03['Area name'].isin(['Tower Hamlets', 'England', 'London'])].filter(regex=r"Males.*"))

In [21]:
[[P03_females.head(), P03_males.head()]]

[[   group  england and wales  london  Tower Hamlets
  0  00-04            1501600  259300           9300
  1  05-09            1634400  260100           8600
  2  10-14            1664100  261700           8600
  3  15-19            1568100  240300           9300
  4  20-24            1701500  304400          16900,
     group  england and wales  london  Tower Hamlets
  0  00-04            1575400  269700           9800
  1  05-09            1714200  271400           9000
  2  10-14            1749000  273600           8800
  3  15-19            1650800  249100           9100
  4  20-24            1712900  284800          15100]]

In [26]:
def population_pyramid(females, males):
    # slider = alt.binding_range(min=1850, max=2000, step=10)
    # select_year = alt.selection_single(name='year', fields=['year'],
                                    #    bind=slider, init={'year': 2000})

    females = alt.Chart(females).properties(
        width=250
    )
    males = alt.Chart(males).properties(
        width=250
    )

    # color_scale = alt.Scale(domain=['Male', 'Female'],
    #                         range=['#1f77b4', '#e377c2'])

    left = females.encode(
        y=alt.Y('group:O', axis=None),
        x=alt.X('Tower Hamlets:Q',
                title='2021 population',
                sort=alt.SortOrder('descending')),
        # color=alt.Color('gender:N', scale=color_scale, legend=None)
    ).mark_bar().properties(title='Tower Hamlets females')

    middle = females.encode(
        y=alt.Y('group:O', axis=None),
        text=alt.Text('group:O'),
    ).mark_text().properties(width=20)

    right = males.encode(
        y=alt.Y('group:O', axis=None),
        x=alt.X('Tower Hamlets:Q',
                title='2021 population'),
        # color=alt.Color('gender:N', scale=color_scale, legend=None)
    ).mark_bar().properties(title='Tower Hamlets males')

    return alt.concat(left, middle, right, spacing=5)


In [27]:
population_pyramid(P03_females, P03_males)

alt.ConcatChart(...)

In [28]:
# P04 Census 2021: Usual resident population density, local authorities in England and Wales
P04 = pd.read_excel(filename,
sheet_name="P04", header=6)

In [29]:
P04[P04['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,Population density (number of usual residents per square kilometre) \n[note 13]
1,E92000001,England,434
211,E12000007,London,5598
224,E09000030,Tower Hamlets,15695


In [31]:
# H01 Census 2021: Number of households with at least one usual resident, local authorities in England and Wales
H01 = pd.read_excel(filename,
sheet_name="H01", header=6)

In [32]:
H01[H01['Area name'].isin(['Tower Hamlets', 'England', 'London'])]

,Area code [note 2],Area name,Number of households with at least one usual resident
1,E92000001,England,23435700
211,E12000007,London,3423800
224,E09000030,Tower Hamlets,120500
